In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
import scipy.cluster.hierarchy as shc
from sklearn import preprocessing
import datetime

%time

# Informasi Dasar yang ingin diambil

In [ ]:
df=pd.read_csv('umkm_data.csv')

In [ ]:
KOLOM=['provinsi','tahun_survey','nama_badan_hukum','jumlah_tenaga_kerja','sistem_penjualan',
       'target_pasar_produk_massal','target_pasar_produk_spesifik','tingkat_pesaing_usaha',
       'total_aset','penjualan_rata-rata_tahunan','kebutuhan_pembiayaan_kredit','nilai_asset_jaminanan',
       'Sektor_Industri','Kategori_jumlah_pinjaman','proyeksi_pertumbuhan_penjualan']

In [ ]:
DATA=df[KOLOM].copy()

# String to numeric

### Kolom rupiah

In [ ]:
kolom_rup=['total_aset','penjualan_rata-rata_tahunan','kebutuhan_pembiayaan_kredit','nilai_asset_jaminanan']

In [ ]:
for kolom in kolom_rup:
    DATA[kolom] = DATA[kolom].str.replace('Rp. ', '').str.replace(' ', '').str.replace('Juta', '')
    DATA[kolom] = DATA[kolom].str.replace(',', '').str.replace('.','')
    DATA[kolom] = DATA[kolom].astype('int64')
    DATA.loc[DATA[kolom]>10**8, kolom] = DATA[DATA[kolom]>10**8][kolom]/10**6
    DATA[kolom]=DATA[kolom] * 10**4

#  penanganan atribut serupa

In [ ]:
def separator(x):
    if 'gol bawah' in x.lower():
        return 'golongan bawah'
    elif 'golongan atas' in x.lower():
        return 'golongan atas'
    elif 'golongan menengah' in x.lower():
        return 'golongan menengah'
    elif 'memenuhi order' in x.lower():
        return 'memenuhi order distributor/exportir/trader'
    elif 'tidak ada'in x.lower():
        return 'tidak ada'
    else:
        return np.NaN

In [ ]:
DATA['sasaran_produk_massal'] = DATA['target_pasar_produk_massal'].apply(separator)
DATA['persentase_massal'] = DATA['target_pasar_produk_massal'].apply(lambda x: str(x.split('\xa0')[-2])[:-3])
DATA['sasaran_produk_spesifik'] = DATA['target_pasar_produk_spesifik'].apply(separator)
DATA['persentase_spesifik'] = DATA['target_pasar_produk_spesifik'].apply(lambda x: str(x.split('\xa0')[-2])[:-3])

DATA.drop('target_pasar_produk_massal', axis=1, inplace=True)
DATA.drop('target_pasar_produk_spesifik', axis=1, inplace=True)

In [ ]:
def sasaran_produk(row):
    if(float(row['persentase_massal'])>=50):
        val=row['sasaran_produk_massal']
    else:
        val=row['sasaran_produk_spesifik']
    return val

In [ ]:
DATA['sasaran_produk']=DATA.apply(sasaran_produk,axis=1)

In [ ]:
DATA.drop('sasaran_produk_massal', axis=1, inplace=True)
DATA.drop('persentase_massal', axis=1, inplace=True)
DATA.drop('sasaran_produk_spesifik', axis=1, inplace=True)
DATA.drop('persentase_spesifik', axis=1, inplace=True)

# Inkonsistensi Data

In [ ]:
DATA['proyeksi_pertumbuhan_penjualan']=DATA.proyeksi_pertumbuhan_penjualan.apply(lambda x:x.split(' %')[0].split('.')[0])
DATA['proyeksi_pertumbuhan_penjualan']=DATA.proyeksi_pertumbuhan_penjualan.replace('-',np.NaN)
DATA['proyeksi_pertumbuhan_penjualan']=DATA.proyeksi_pertumbuhan_penjualan.replace('',np.NaN)
DATA['proyeksi_pertumbuhan_penjualan'] = DATA['proyeksi_pertumbuhan_penjualan'].astype(float)

In [ ]:
#delete tipo
DATA['tahun_survey']=DATA['tahun_survey'].replace(3012,2012)
DATA['tahun_survey']=DATA['tahun_survey'].replace(2913,2013)

#ganti - jadi null
DATA['nama_badan_hukum']=DATA['nama_badan_hukum'].replace('-',np.NaN)

DATA['sistem_penjualan']=DATA['sistem_penjualan'].replace('-',np.NaN)

DATA['tingkat_pesaing_usaha']=DATA['tingkat_pesaing_usaha'].replace('-',np.NaN)

In [ ]:
#ubah tipe data
DATA = DATA.astype({'total_aset':'int64','penjualan_rata-rata_tahunan':'int64',
                    "kebutuhan_pembiayaan_kredit":'int64','nilai_asset_jaminanan':'int64'})

## DATA IMBALANCE

### Badan Hukum

In [ ]:
DATA = DATA[DATA['nama_badan_hukum'].isin(['Perorangan', 'Usaha Dagang (UD)', np.NaN])]

In [ ]:
DATA.drop('nama_badan_hukum', axis=1, inplace=True)

### Sektor industri

In [ ]:
sektor = pd.DataFrame(DATA['Sektor_Industri'].value_counts()>100).reset_index()
sektorbesar = sektor[sektor['Sektor_Industri'] == True]['index'].tolist()
DATA = DATA[DATA['Sektor_Industri'].isin(sektorbesar)]

In [ ]:
def sektor_industri(row):
    if row['Sektor_Industri']=='Perdagangan Besar dan Eceran Reparasi dan Perawatan Mobil dan Sepeda Motor':
        val='Perdagangan dan Perawatan Kendaraan'
    elif row['Sektor_Industri']=='Industri Pengolahan':
        val='Industri Pengolahan'
    elif row['Sektor_Industri']==np.NaN:
        val=np.NaN
    else:
        val='Lain-lain'
    return val

In [ ]:
DATA['Sektor_Industri']=DATA.apply(sektor_industri,axis=1)

# CAT -> angka

In [ ]:
# ordinal
# tingkat_pesaing_usaha, Kategori_jumlah_pinjaman, sasaran_produk

# nominal
# nama_badan_hukum, sektor_industri, sistem_penjualan

## ordinal

In [ ]:
DATA = DATA.replace({
    'tingkat_pesaing_usaha' : { 'Rendah' : 2, 'Sedang' : 1, 'Tinggi' : 0},
    'sasaran_produk' : { 'tidak ada' : 0, 'golongan bawah' : 1, 'golongan menengah' : 2, 'golongan atas' : 3,
                        'memenuhi order distributor/exportir/trader' : 4},
    'Kategori_jumlah_pinjaman' : {'s.d Rp 10 JUTA': 0, 'Rp 10 JUTA s.d Rp 50 JUTA':1,
                                  'Rp 50 JUTA s.d Rp 100 JUTA':2, '> Rp 100 JUTA':3}
})

## Nominal

### Sistem Penjualan

In [ ]:
# membuat hanya 2 sistem penjualan, menjual kepada retail atau tidak. jika ya = 1, tidak = 0.

In [ ]:
DATA = DATA.replace({'sistem_penjualan' : { 'Retail' : 1, 'Retail dan Distributor' : 1, 'Distributor' : 0}})

# Missing Value

### berdasarkan provinsi + tahun

In [ ]:
DATA.dropna(subset=['provinsi'],axis=0,inplace=True)
DATA.drop(['provinsi','tahun_survey'],axis=1,inplace=True)

In [ ]:
DATA.dropna(subset=['sistem_penjualan', 'tingkat_pesaing_usaha'],axis=0,inplace=True)

In [ ]:
DATA.dropna(subset=['proyeksi_pertumbuhan_penjualan'],axis=0,inplace=True)

## DATA extreme

In [ ]:
DATA=DATA[DATA['total_aset']!=0]
DATA=DATA[DATA['penjualan_rata-rata_tahunan']!=0]
DATA=DATA[DATA['kebutuhan_pembiayaan_kredit']!=0]

In [ ]:
DATA.to_csv('handling.csv', index=False)

# KELAR DATA HANDLING

In [ ]:
DATA = pd.read_csv('handling.csv')

<!-- ## boxplot outlier -->

In [ ]:
# DATA = DATA.reset_index(drop=True)
# DATA['log_total_aset'] = np.log10(DATA['total_aset'])
# DATA['log_penjualan_rata-rata_tahunan'] = np.log10(DATA['penjualan_rata-rata_tahunan'])
# DATA['log_kebutuhan_pembiayaan_kredit'] = np.log10(DATA['kebutuhan_pembiayaan_kredit'])
# DATA['log_nilai_asset_jaminanan'] = np.log10(DATA['nilai_asset_jaminanan'])

In [ ]:
# DATA['log_total_aset']=DATA['log_total_aset'].replace(float('-inf'),0)
# DATA['log_penjualan_rata-rata_tahunan']=DATA['log_penjualan_rata-rata_tahunan'].replace(float('-inf'),0)
# DATA['log_kebutuhan_pembiayaan_kredit']=DATA['log_kebutuhan_pembiayaan_kredit'].replace(float('-inf'),0)
# DATA['log_nilai_asset_jaminanan']=DATA['log_nilai_asset_jaminanan'].replace(float('-inf'),0)

In [ ]:
# sns.boxplot(DATA['log_kebutuhan_pembiayaan_kredit'])

In [ ]:
# DATA.boxplot(column=['log_total_aset', 'log_penjualan_rata-rata_tahunan', 'log_kebutuhan_pembiayaan_kredit'])

# ANALISA

## jadiin log

In [ ]:
DATA = DATA.reset_index(drop=True)
DATA['log_total_aset'] = np.log10(DATA['total_aset'])
DATA['log_penjualan_rata-rata_tahunan'] = np.log10(DATA['penjualan_rata-rata_tahunan'])
DATA['log_kebutuhan_pembiayaan_kredit'] = np.log10(DATA['kebutuhan_pembiayaan_kredit'])
DATA['log_nilai_asset_jaminanan'] = np.log10(DATA['nilai_asset_jaminanan'])

In [ ]:
DATA['log_nilai_asset_jaminanan']=DATA['log_nilai_asset_jaminanan'].replace(float('-inf'),0)

# REKAYASA FITUR

## rasio efisiensi + jaminan

In [ ]:
DATA['rasio_efisiensi'] = DATA['log_penjualan_rata-rata_tahunan']/DATA['log_total_aset']
DATA['rasio_jaminan'] = DATA['log_nilai_asset_jaminanan']/DATA['log_kebutuhan_pembiayaan_kredit']

In [ ]:
# DATA['rasio_efisiensi'] = DATA['penjualan_rata-rata_tahunan']/DATA['total_aset']
# DATA['rasio_jaminan'] = DATA['nilai_asset_jaminanan']/DATA['kebutuhan_pembiayaan_kredit']

## jenis umkm 

In [ ]:
def jenis_umkm(row):
    if(row['total_aset']<=50*10**6):
        val='Mikro'
    elif((row['total_aset']>50*10**6) & (row['total_aset']<=500*10**6)):
        val='Kecil'
    elif((row['total_aset']>500*10**6) & (row['total_aset']<=10*10**9)):
        val='Menengah'
    else:
        val= np.NaN
    return val

In [ ]:
DATA['jenis_umkm']=DATA.apply(jenis_umkm,axis=1)

In [ ]:
DATA.dropna(inplace=True)

In [ ]:
DATA.to_csv('data_logbenergan.csv',index=False)

# SELEKSI FITUR

In [ ]:
DATA.columns

In [ ]:
seleksi = DATA[['jumlah_tenaga_kerja', 'sistem_penjualan', 'tingkat_pesaing_usaha','Sektor_Industri',
            'Kategori_jumlah_pinjaman','proyeksi_pertumbuhan_penjualan', 'sasaran_produk',
            'log_total_aset','log_penjualan_rata-rata_tahunan','log_kebutuhan_pembiayaan_kredit',
            'log_nilai_asset_jaminanan', 'rasio_efisiensi', 'rasio_jaminan', 'jenis_umkm']]

In [ ]:
seleksi.drop('jenis_umkm', axis=1,inplace=True)

In [ ]:
skenario1 = pd.get_dummies(seleksi, columns=['Sektor_Industri'], drop_first=True)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.heatmap(skenario1.corr(),annot=True)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=30,
    horizontalalignment='right'
)

## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
skenario2 = seleksi[['jumlah_tenaga_kerja', 'log_total_aset','log_penjualan_rata-rata_tahunan',
                     'log_kebutuhan_pembiayaan_kredit', 'log_nilai_asset_jaminanan', 'rasio_efisiensi',
                     'rasio_jaminan', 'proyeksi_pertumbuhan_penjualan']]

In [ ]:
korelasi = skenario2.corr()

In [ ]:
for i in korelasi.columns:
    temp=abs(korelasi[i])
    col=temp[(temp>0.45) & (temp!=1)].index
    if col.empty:
        a=0
    else:
        print(i,col)

## ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
skenario3 = seleksi[['log_kebutuhan_pembiayaan_kredit', 'proyeksi_pertumbuhan_penjualan',
                     'rasio_jaminan', 'rasio_efisiensi']]

In [ ]:
korelasi = skenario3.corr()

In [ ]:
for i in korelasi.columns:
    temp=abs(korelasi[i])
    col=temp[(temp>0.47) & (temp!=1)].index
    if col.empty:
        a=0
    else:
        print(i,col)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.heatmap(skenario3.corr(),annot=True)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=30,
    horizontalalignment='right'
)
ax.set_title('Non-Collinearity Attributes ', fontsize=16)

In [ ]:
data = skenario3

In [ ]:
data.log_kebutuhan_pembiayaan_kredit

# Data Scaling

In [ ]:
x_array = np.array(data)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(x_array)

# Clustering Model

## hyperparameter tuning

In [ ]:
def tuningKmeans(data,maxn,inits):
    best_score=-9999999
    waktu=datetime.datetime.now()
    namafile=str(waktu.strftime("%Y%m%d-%H%M%S"))+'_log_'+str(maxn)+str(inits)+'kmeans.txt'
    f = open(namafile,'w',encoding="utf-8")
    for init in inits:
        for i in range(2,maxn+1):
#             print(i,init)
            km = KMeans(n_clusters=i, random_state=100,init=init)
            pred = km.fit_predict(data)
            silhouette_avg = silhouette_score(data, pred)
            out='Silhouette score :'+str(silhouette_avg)+' dengan n='+str(i)+' tipe init= '+str(init)+'\n'
            print(out)
            f.write(out)
            if(silhouette_avg>best_score):
                best_score=silhouette_avg
                best_model=km
                best_pred=pred
                best_n=i
                best_init=init
    out='Best Model didapatkan dari n='+str(best_n)+', dengan init='+str(best_init)+', dengan score='+str(best_score)+'\n'
    print(out)
    f.write(out)
    f.close()
    return(best_model,best_pred)

In [ ]:
maxn=5
inits=['k-means++','random']
model,pred=tuningKmeans(data_scaled,maxn,inits)

In [ ]:
def tuningHierar(data,maxn,linkages):
    best_score=-9999999
    waktu=datetime.datetime.now()
    namafile=str(waktu.strftime("%Y%m%d-%H%M%S"))+'_log_'+str(maxn)+str(linkages)+'AggloClust.txt'
    f = open(namafile,'w',encoding="utf-8")
    for linkage in linkages:
        for i in range(2,maxn+1):
    #             print(i,init)
            hie=AgglomerativeClustering(n_clusters=i,linkage=linkage)
            pred = hie.fit_predict(data)
            silhouette_avg = silhouette_score(data, pred)
            out='Silhouette score :'+str(silhouette_avg)+' dengan n='+str(i)+' tipe linkage= '+str(linkage)+'\n'
            print(out)
            f.write(out)
            if(silhouette_avg>best_score):
                best_score=silhouette_avg
                best_model=hie
                best_pred=pred
                best_n=i
                best_linkage=linkage
    out='Best Model didapatkan dari n='+str(best_n)+', dengan score='+str(best_score)+' tipe linkage= '+str(best_linkage)+'\n'
    print(out)
    f.write(out)
    f.close()
    return(best_model,best_pred)

In [ ]:
maxn=5
linkages=['ward','complete']
model,pred=tuningHierar(data_scaled,maxn,linkages)

## evaluasi model

In [ ]:
n=3

In [ ]:
%%time
hie = AgglomerativeClustering(n_clusters=2,linkage='complete')
pred_h = hie.fit_predict(data_scaled)
silhouette_avg = silhouette_score(data_scaled, pred_h)
print(silhouette_avg)

In [ ]:
%%time
km = KMeans(n_clusters=n, random_state=0,init='k-means++')
pred_p = km.fit_predict(data_scaled)
silhouette_avg = silhouette_score(data_scaled, pred_p)
print(silhouette_avg)

In [ ]:
%%time
km = KMeans(n_clusters=n, random_state=0,init='random')
pred_r = km.fit_predict(data_scaled)
silhouette_avg = silhouette_score(data_scaled, pred_r)
print(silhouette_avg)

### perbandingan korelasi

In [ ]:
korr = data.copy()

In [ ]:
korr['Class_p']= pred_p
korr['Class_r']= pred_r
korr['Class_h']= pred_h

In [ ]:
korelasi = korr.corr(method='spearman')

In [ ]:
corr_p=abs(korelasi['Class_p'])

In [ ]:
corr_p

In [ ]:
corr_h=abs(korelasi['Class_h'])

In [ ]:
corr_h

In [ ]:
corr_r=abs(korelasi['Class_r'])

In [ ]:
corr_r

## Multi class Clustering

Mikro

In [ ]:
dfmikro = data[data['jenis_umkm'] == 'Mikro'].drop('jenis_umkm', axis=1)
# dfkecil = data[data['jenis_umkm'] == 'Kecil'].drop('jenis_umkm', axis=1)
# dfmenengah = data[data['jenis_umkm'] == 'Menengah'].drop('jenis_umkm', axis=1)

In [ ]:
x_array = np.array(dfmikro)
# x_array = np.array(dfkecil)
# x_array = np.array(dfmenengah)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(x_array)